In [ ]:
import numpy as np
import re
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
f = open("/content/drive/My Drive/book-war-and-peace.txt")
t = f.readlines()
t = ''.join(t)

In [ ]:
text = re.sub('[^a-zA-Z]', ' ', t)[:50000]
text = text.lower()

In [ ]:
d = {}
for i in range(0, 26):
    d[chr(i+97)] = i
d[' '] = 26

In [ ]:
V = np.zeros(50000)
for i in range(len(text)):
    V[i] = d[text[i]]

In [ ]:
V = V[0:100].astype(int)
print(V)

[ 2  7  0 15 19  4 17 26  8 26 26 26 22  4 11 11 26 26 15 17  8 13  2  4
 26 26 18 14 26  6  4 13 14  0 26  0 13  3 26 11 20  2  2  0 26  0 17  4
 26 13 14 22 26  9 20 18 19 26  5  0 12  8 11 24 26  4 18 19  0 19  4 18
 26 14  5 26 19  7  4 26  1 20 14 13  0 15  0 17 19  4 18 26 26  1 20 19
 26  8 26 22]


In [ ]:
a = np.array([[0.47468,0.52532],[0.51656,0.48344]])
print(a)

[[0.47468 0.52532]
 [0.51656 0.48344]]


In [ ]:
b = np.array([[0.03735,0.03909 ],
[0.03408 ,0.03537 ],
[0.03455 ,0.03537 ],
[0.03828, 0.03909 ],
[0.03782, 0.03583 ],
[0.03922 ,0.03630 ],
[0.03688, 0.04048 ],
[0.03408, 0.03537 ],
[0.03875 ,0.03816 ],
[0.04062 ,0.03909 ],
[0.03735 ,0.03490 ],
[0.03968, 0.03723 ],
[0.03548 ,0.03537 ],
[0.03735 ,0.03909 ],
[0.04062, 0.03397 ],
[0.03595, 0.03397 ],
[0.03641, 0.03816 ],
[0.03408, 0.03676 ],
[0.04062 ,0.04048 ],
[0.03548, 0.03443 ],
[0.03922, 0.03537 ],
[0.04062 ,0.03955], 
[0.03455, 0.03816 ],
[0.03595, 0.03723 ],
[0.03408,0.03769 ],
[0.03408 ,0.03955 ],
[0.03688, 0.03397 ]])
b = b.T
print(b.shape)

(2, 27)


In [ ]:
pi = np.array([ 0.51316,0.48684])
print(pi[0],pi[1])

0.51316 0.48684


In [ ]:
print(V.shape)
print(a.shape)
print(b.shape)
print(V.shape[0], a.shape[0])

(100,)
(2, 2)
(2, 27)
100 2


In [ ]:
N = 2
M = 27
MaxIters = 100
iters = 0
import math
oldLogProb = -math.inf

In [ ]:
T = len(V)  
print(T)

100


In [ ]:
def War_peace(a,b,V,pi,N,M,T,oldLogProb):
# Alpha pass
    c = np.zeros([T, 1])
    alpha = np.zeros([T,N])

    for i in range(N):
        alpha[0][i] = pi[i]*b[i][V[0]]
        c[0][0] = c[0][0]+alpha[0][i]
    c[0][0] = 1/c[0][0] 

    for i in range(N):
        alpha[0][i] = c[0][0]*alpha[0][i]

    for t in range(1,T):
        c[t][0] = 0
        for i in range(N):
            alpha[t][i]=0
            for j in range(N):
                alpha[t][i] = alpha[t][i]+ alpha[t-1][i]*a[j][i]
            alpha[t][i] = alpha[t][i]*b[i][V[t]]
            c[t][0] = c[t][0] + alpha[t][i]        


        c[t][0] = 1/ c[t][0]


        for i in range(N):
            alpha[t][i] = c[t][0]*alpha[t][i]

    # print(c)
    beta = np.zeros([T,N])
    for i in range(N):
        beta[T-1][i] = c[T-1][0]

    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t][i] = 0
            for j in range(N):
                beta[t][i] = beta[t][i]+ a[i][j]*b[j][V[t+1]]*beta[t+1][j]
            beta[t][i] = c[t][0]*beta[t][i]

                  

    digamma=  np.zeros([T,N,N])
    gamma =  np.zeros([T,N])
    for t in range(T-1):
        for i in range(N):
            gamma[t][i]=0
            for j in range(N):
                digamma[t][i][j] = alpha[t][i]*a[i][j]*b[j][V[t+1]]*beta[t+1][j]
                gamma[t][i] = gamma[t][i]+digamma[t][i][j]  


    for i in range(N):
         gamma[T-1][i]= alpha[T-1][i]
    # print(gamma)
    # re-estimate π
    for i in range(N):
        pi[i] = gamma[0][i]
    #re-estimate A
    for i in range(N):
        denom = 0
        for t in range(T-1):
            denom = denom + gamma[t][i]
        for j in range(N):
            numer = 0
            for t in range(T-1):
                numer = numer + digamma[t][i][j]
            a[i][j] = numer/denom

    #re-estimate B
    for i in range(N):
        denom = 0
        for t in range(T):
            denom = denom + gamma[t][i]
        for j in range(M):
            numer = 0
            for t in range(T):
                if (V[t]==j):
                    numer = numer + gamma[t][i]
            b[i][j] = numer/denom
    import math
    logprob = 0
    for i in range(T):
        logprob = logprob + math.log(c[i][0])
    logprob = -1*logprob
    
    if logprob>oldLogProb:
        oldLogProb = logprob
#             War_peace(a,b,V,pi,N,M,T,oldLogProb)
            
       
    return a,b,pi,oldLogProb






In [ ]:
A,B,pie,P = War_peace(a,b,V,pi,N,M,T,oldLogProb)
for i in range(5):
    A,B,pie,P =  War_peace(A,B,V,pie,N,M,T,P)
    


print('A',A)
print('B',B)
print('PI',pie)
print("log(p(0/lamda))",P)


A [[0.01734974 0.98265026]
 [0.56146022 0.43853978]]
B [[1.67586753e-002 9.86942861e-035 9.60492341e-001 2.79661012e-018
  6.53789347e-007 2.53096469e-019 4.89623116e-018 2.26526483e-002
  4.64694128e-011 6.48108851e-019 0.00000000e+000 1.67394042e-011
  2.99003392e-019 8.94369829e-018 5.58184995e-018 9.03032866e-005
  0.00000000e+000 4.26494542e-008 4.83577164e-018 5.29712552e-006
  9.22540628e-018 0.00000000e+000 3.75688832e-011 0.00000000e+000
  1.21482589e-021 0.00000000e+000 3.79498347e-008]
 [1.63056977e-003 2.10925637e-131 7.19823344e-004 1.18127030e-034
  1.04803982e-001 5.37227537e-095 7.53118434e-017 1.27789253e-003
  1.98635605e-001 7.93809263e-068 0.00000000e+000 3.39570713e-002
  7.04884818e-104 4.78935490e-004 7.19084006e-012 2.52627956e-002
  0.00000000e+000 8.29955085e-002 5.97264830e-008 8.08691571e-003
  8.17655549e-047 0.00000000e+000 4.08900114e-002 0.00000000e+000
  8.59924570e-111 0.00000000e+000 5.01260830e-001]]
PI [1.23020247e+48 5.11115425e+47]
log(p(0/lamda))

In [ ]:
A,B,pie,P = War_peace(a,b,V,pi,N,M,T,oldLogProb)
for i in range(5):
    A,B,pie,P =  War_peace(A,B,V,pie,N,M,T,P)
    


print('A',A)
print('B',B)
print('PI',pie)
print("log(p(0/lamda))",P)


A [[nan nan]
 [nan nan]]
B [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan]]
PI [nan nan]
log(p(0/lamda)) -2162.7708036739004


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in double_scalars
